In [1]:
import pandas as pd
import numpy as np
import math

from keras.layers import Embedding, Dense, LSTM, Dense, Input, concatenate
from keras.models import Model
from keras.utils import plot_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

Using TensorFlow backend.


In [2]:
bot_accounts = pd.concat([pd.read_csv('social_spambots_1.csv'), pd.read_csv('social_spambots_2.csv'), pd.read_csv('social_spambots_3.csv')]).reset_index(drop=True)
clean_accounts = pd.read_csv('geniune_accounts.csv')

requiredColumns = ['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'protected', 'default_profile']
bot_accounts = bot_accounts[requiredColumns]
clean_accounts = clean_accounts[requiredColumns]
def clean_df(df):
    
    
    df['listed_count'] = df['listed_count'].apply(lambda x: x if x==x else 0)
    df['geo_enabled'] = df['geo_enabled'].apply(lambda x: 1 if x==x else 0)
    df['statuses_count'] = df['statuses_count'].apply(lambda x: x if x==x else 0)
    df['friends_count'] = df['friends_count'].apply(lambda x: x if x==x else 0)
    df['followers_count'] = df['followers_count'].apply(lambda x: x if x==x else 0)
    df['favourites_count'] = df['favourites_count'].apply(lambda x: x if x==x else 0)
    df['profile_use_background_image'] = df['profile_use_background_image'].apply(lambda x: 1 if x==x else 0)
    df['verified']=df['verified'].apply(lambda x: 1 if x==x else 0)
    df['is_protected']=df['protected'].apply(lambda x: 1 if x==x else 0)
    df['default_profile'] = df['default_profile'].apply(lambda x: 0 if x==x else 1)
    #df = df.rename(index=str, columns={"screen_name": "username", "statuses_count": "total_tweets", "friends_count": "total_following", "followers_count": "total_followers", "favourites_count": "total_likes"})
    return df[['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'is_protected', 'default_profile']]

bot_accounts = clean_df(bot_accounts)
clean_accounts = clean_df(clean_accounts)
#print(bot_accounts)

    



C:\Users\STUDENT\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [3]:
bot_accounts['BotOrNot'] = 1
clean_accounts['BotOrNot'] = 0
#print(bot_accounts)
#print(clean_accounts)

combined_df = pd.concat([bot_accounts, clean_accounts])

new_df = combined_df.sample(frac=1).reset_index(drop=True)

In [4]:
training_df = new_df[:int(combined_df.shape[0] * 0.8)]
test_df = new_df[int(combined_df.shape[0] * 0.8):]



columns_to_standardize = ['listed_count', 'geo_enabled', 'verified', 'statuses_count', 'friends_count','followers_count', 'favourites_count', 'profile_use_background_image', 'is_protected', 'default_profile']

training_df_mean = training_df[columns_to_standardize].mean()
training_df_std = training_df[columns_to_standardize].std()

training_df[columns_to_standardize] = (training_df[columns_to_standardize] - training_df_mean)/training_df_std
test_df[columns_to_standardize] = (test_df[columns_to_standardize] - training_df_mean)/training_df_std

# training_df_mean = training_df.mean()
# training_df_std = training_df.std()

# training_df = (training_df - training_df_mean)/training_df_std
# test_df = (test_df - training_df_mean)/training_df_std

# max_vals = training_df.max()

# training_df = training_df/max_vals
# test_df = test_df/max_vals

C:\Users\STUDENT\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [10]:
#X_train = training_df.drop(['BotOrNot', 'is_protected'], axis=1).values
#y_train = training_df['BotOrNot'].values.reshape(-1,1)
#print(X_train)
X_train = training_df
y_train = training_df['BotOrNot'].values.reshape(-1,1)

X_test = test_df
y_test = test_df['BotOrNot'].values.reshape(-1,1)
print()
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN
sm = SMOTEENN()
X_train, y_train = sm.fit_sample(X_train, y_train)
#print(X_train)
from sklearn.ensemble import AdaBoostClassifier

# Import Support Vector Classifier
from sklearn.svm import SVC
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
svc=SVC(probability=True, kernel='linear')

# Create adaboost classifer object
abc =AdaBoostClassifier(n_estimators=50, base_estimator=svc,learning_rate=1)

# Train Adaboost Classifer
model = abc.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = model.predict(X_test)


# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

C:\Users\STUDENT\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\STUDENT\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.9964243146603099


In [6]:
inp = Input(shape=[11])

another = Dense(500, activation='relu')(inp)
another = Dense(200, activation='relu')(another)
another = Dense(1, activation='sigmoid')(another)

mod = Model(inp, another)
mod.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
training = mod.fit(x=X_train, y=y_train, batch_size=200, epochs=20, validation_data=(X_test, y_test))

Train on 7798 samples, validate on 1678 samples
Epoch 1/20
7798/7798 [==============================] - 0s 14us/step - loss: 6.5361e-07 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 0.9982
Epoch 2/20
7798/7798 [==============================] - 0s 13us/step - loss: 6.3727e-07 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 0.9982
Epoch 3/20
7798/7798 [==============================] - 0s 12us/step - loss: 6.2008e-07 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 0.9982
Epoch 4/20
7798/7798 [==============================] - 0s 13us/step - loss: 6.0452e-07 - acc: 1.0000 - val_loss: 0.0023 - val_acc: 0.9982
Epoch 5/20
7798/7798 [==============================] - 0s 12us/step - loss: 5.8971e-07 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 0.9982
Epoch 6/20
7798/7798 [==============================] - 0s 13us/step - loss: 5.7481e-07 - acc: 1.0000 - val_loss: 0.0022 - val_acc: 0.9982
Epoch 7/20
7798/7798 [==============================] - 0s 13us/step - loss: 5.5960e-07 - acc: 1.0000 - val_loss: 0.00

In [67]:
mod.save('my_model.h5')

In [10]:
print(training_df_mean)

listed_count                       9.997615
geo_enabled                        0.259392
verified                           0.001640
statuses_count                  8055.425462
friends_count                    614.043381
followers_count                  998.432469
favourites_count                1983.179487
profile_use_background_image       0.952296
is_protected                       0.008945
default_profile                    0.840936
dtype: float64
